# Insulators and metals

In this notebook we discuss a few standard cases of materials to model. Modelling different materials will required different numerical setups, which is what we will explore here.


## Insulators: Modelling silicon

The simplest type of material to model are insulators. We will start with silicon.

In [ ]:
using DFTK
using ASEconvert

# Use the atomistic simulation environment (ASE) to look up the structure of bulk silicon
# Since ASE is a python package, we need to explicitly convert to AbstractSystem,
# the data type used for chemical systems in Julia
silicon = pyconvert(AbstractSystem, ase.build.bulk("Si"))

# Attach pseudopotentials (model for electron-nuclear interaction)
system = attach_psp(silicon; Si="hgh/lda/si-q4")

# Setup an LDA model in DFTK
model = model_LDA(system)

# Discretise the problem in a plane-wave basis
# We use a mesh of 2x2x2 k-Points in the Brillouin zone
# and an energy cutoff of 15 Hartree
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[2, 2, 2])

# Run the SCF and catch the result:
scfres = self_consistent_field(basis, tol=1e-6);

We also plot a bandstructure, i.e. perform a plot of the eigenvalues of the $k$-point specific Hamiltonian $H_k$ versus $k$.

In [ ]:
using Plots
plot_bandstructure(scfres)

We notice a clear gap between the bands around the Fermi level (the energy zero in the plot above). This is the defining property of an insulator.

## Metals: Modelling aluminum

**Exercise:** Try running a simulation of aluminium yourself.

For aluminum a possible structural setup can again be looked up using `ASEconvert`:
```julia
aluminium = pyconvert(AbstractSystem, ase.build.bulk("Al", cubic=true))
system = attach_psp(aluminium; Al="hgh/lda/al-q3")
```

First use the same LDA model as before, defined using the DFTK shortcut
```julia
model = model_LDA(system)
```
and use `Ecut=15` and `kgrid=[2, 2, 2]` to discretise the problem.

When running the SCF like this you should observe either slow or no convergence.
This points to the fact that modelling aluminium is clearly a little different
than silicon.
In fact the issue is due to aluminium being a metal,
while silicon is an insulator / semiconductor.
More on this below.

To make progress and achieve SCF convergence add the parameter
```julia
model = model_LDA(system; temperature=1e-3, smearing=Smearing.Gaussian())
```
This is called **smearing** and is a numerical trick
to improve the convergence for quantities which involve
sums over $k$-Points in metals.
As the density $\rho$ involves such a sum, the SCF typically fails
to converge in metals without smearing.

With this setup it the SCF should succeed. Use the obtained scfres to compute the band structure of aluminium with `plot_bandstructure(scfres)`.

In [ ]:
using DFTK
using ASEconvert

aluminium = pyconvert(AbstractSystem, ase.build.bulk("Al"))
system = attach_psp(aluminium; Al="hgh/lda/al-q3")

# your solution here 

## Contrasting aluminium and silicon

With these two calculations we have computed two band structures,
one for a metal (aluminium) and one for a semiconductor (silicon).

- All details of the band structure do not matter for our purposes.
- The important difference is that silicon has a so-called **band gap**,
  i.e. a region near the Fermi level (which is at $0$ in the shown band plots)
  where there are no eigenvalues, i.e. no blue bands.
  In contrast aluminium does not have this gap.
- This makes aluminium a metal and silicon an insulator respectively semiconductor.

For the numerical treatment the absence of a band gap has strong implications:
- At the beginning of the SCF the DFT Hamiltonian $H[\rho_n]$
  at the current density $\rho_n$ is only a rough approximation to the
  converged Hamiltonian $H[\rho_\ast]$.
- This means that the order of the $\psi_{ki}$ near the Fermi level,
  i.e. near $i = N$, may be easily swapped as many $\psi_{ki}$
  of similar eigenvalues $\varepsilon_i$ are close.
- From one iteration to the next the $\psi_{ki}$ which are summed
  over in the density expression
  $$ \rho = \sum_{k\in\text{BZ}} \sum_{i=1}^N 2 |\psi_{ki}|^2 $$
  may thus change (as $N$ becomes $N+1$, thus not summed over, but $N+1$ becomes $N$).
- Without going into details the smearing temperature reduces this
  problem by not enforcing a hard cut between the functions summed over in the density,
  but employing a smoother cutoff function.

## K-Point sampling versus supercells

We consider the following aluminium setup, which allows to easily adjust the Monkhorst-Pack grid as well as construct a supercell of the primitive aluminium system.

In [ ]:
using DFTK
using ASEconvert

function aluminium_setup(; Ecut=10.0, kgrid=(1, 1, 1), supercell=(1, 1, 1))
    # Use ASE to make an appropriate aluminium supercell
    pysystem = ase.build.bulk("Al") * pytuple(supercell)
    
    # Convert to AbstractSystem and attach pseudopotentials:
    aluminium = pyconvert(AbstractSystem, pysystem)
    system = attach_psp(aluminium; Al="hgh/lda/al-q3")

    # Construct an PBE model and discretise
    model = model_PBE(system; temperature=1e-2, smearing=Smearing.Gaussian())
    PlaneWaveBasis(model; Ecut, kgrid)
end

As discussed in lectures, modelling supercells of the system is exactly equivalent to using a particular Monkhorst-Pack $k$-point grid. We will now show this numerically.

**Exercise:**
Run a `(4, 1, 1)` supercell calculation using a single $k$-point and a tight density convergence (`tol=1e-8`) and record the final energy per atom in the unit cell. Repeat the calculation for a unit cell `supercell=(1, 1, 1)` and `kgrid=(4, 1, 1)` and again record the *energy per atom* (keeping in mind that the cells contain a different number of aluminium atoms). What do you notice? Check whether this result is dependent on supercell choice or the plane-wave discretisation.

In [ ]:
# Your solution here ... 

## Modelling isolated systems

DFT codes based on plane-wave discretisations (such as DFTK) are by nature most ideally suited for problems with periodic boundary conditions. However, this does not mean that isolated systems such as molecules or atoms cannot be modelled at all. The main idea is to place such an isolated system into a (usually cubic) box of sufficient size, such that the system cannot interact with itself (across the boundary of the box and into one of the other periodic replica).

We will explore this by modelling a neon atom using the following setup:

In [ ]:
using DFTK

a = 4.0
lattice = a * Matrix(I, 3, 3)  # Cubic lattice of length a in each dimension
atoms = [ElementPsp(:Ne, psp=load_psp("hgh/pbe/ne-q8"))]
positions = [zeros(3)]  # Place atom into the middle of the box.

model = model_PBE(lattice, atoms, positions)

# For isolated systems it only makes sense to model the Gamma point (i.e. take kgrid=(1, 1, 1) )
basis = PlaneWaveBasis(model; Ecut=15, kgrid=(1, 1, 1))

scfres = self_consistent_field(basis; tol=1e-7);

**Exercise:**

(a) A lattice constant of $a = 4.0$ Bohr is far too small to avoid spurious interactions of the neon atom with itself across the periodic boundary. Increase $a$ in steps of $1.0$ and monitor the convergence by plotting the energy differences on a semilog plot. What sizes are needed to avoid spurious interactions?

(b) Modelling isolated atoms with DFT can be tricky if $p$-, $d$- or $f$-shells are half filled. To explore this consider the isolated oxygen atom. The setup is as above, but using
```julia
atoms = [ElementPsp(:O, psp=load_psp("hgh/pbe/o-q6"))]
```
   You should also add a little temperature (i.e. add `temperature=1e-3` to the arguments passed to `model_PBE`). What do you observe?

## Modelling magnetic systems: Iron

Only few compounds exhibit a natural permanent magnetism. One of these is iron, while most (like silicon) are not magnetic. This exercise tries to provide a little insight how one could understand, using DFT simulations, why this is the case.

The key assumption in this exercise will be that DFT is a realistic model and that the SCF therefore finds a good approximation to the true ground state of a compound. If this ground state turns out to be magnetic, the compound should therefore feature a permanant magnetism.

We use this computational setup to simulate silicon:

In [ ]:
using DFTK

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
atoms = [Si, Si]
positions = [-ones(3)/8, ones(3)/8]
# Note: This is essentially the same structure as the one produced by
#     silicon = pyconvert(AbstractSystem, ase.build.bulk("Si"))
# above, just manually specifying lattice, atoms and positions.

# Guess some inital magnetic moments
# (Need to be != 0 otherwise SCF makes the assumption that the ground state is not magnetic
#  to gain some performance ...)
magnetic_moments = [2, 2]

model  = model_LDA(lattice, atoms, positions; temperature=0.01, magnetic_moments)
basis  = PlaneWaveBasis(model; Ecut=13, kgrid=[2, 2, 2]);
ρ0     = guess_density(basis, magnetic_moments)
scfres = self_consistent_field(basis; ρ=ρ0);
scfres.energies.total

Even though we have forced some magnetism into the initial density guess, this magnetisation (indicated by `Magnet`) disappears as the SCF converges. Therefore silicon appears to have a non-magnetic ground state.

**Exercise:**  

(i) Repeat the calculation for iron. In this case the system setup is
```julia
a = 5.42352  # iron lattice constant in bohr
lattice = a / 2 * [[-1  1  1];
                   [ 1 -1  1];
                   [ 1  1 -1]]
Fe = ElementPsp(:Fe, psp=load_psp("hgh/lda/Fe-q8.hgh"))
atoms = [Fe]
positions = [zeros(3)]
magnetic_moments = [3]
```
otherwise use the same settings as in the silicon calculation. Based on this calculation, what would you conclude?

(ii) As it turns out too small basis sets and Brilouin-zone sampling (too small `kgrid`s) are not able to support magnetic ground states. Repeat both the silicon as well as the iron calculations for different values of `Ecut` and `kgrid` (i.e. `[1,1,1]`,`[3,3,3]`, `[4,4,4]` etc.), where in both cases larger means better. Play with these parameters to determine the first two digits of the ground state energy of silicon and iron. Based on these numerical parameters what do you conclude now?

(iii) To show that a non-magnetic iron structure is actually higher in energy re-run the iron calculation with the `Ecut` and `kgrid` parameters determined in (ii), but this time set `magnetic_moments = [0]`. This enforces the SCF to converge to a non-magnetic ground state. This is why the magnetisation `Magnet` is no longer printed ... it is `0` by assumption. What is the energy difference between this non-magnetic iron ground state and the ground state you determined in (ii)?

#### Takeaways
- Insulators / semiconductors have a band gap
- Metals have no band gap
- Without a band gap (for metals) a smearing is needed to aid convergence


- Obtaining properties accurately requires a sufficient discretisation setup. E.g. one requently mischaractarises a material (metal versus insulator, magnetism etc) if the setup is not sufficiently fine.